In [ ]:
from matplotlib import pyplot as plt
import random
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
# a=np.array([3,7.6]) #correct angle to x axis=68.5 degrees
# #usage of ::-1 operator
# m_false=np.arctan2(*a)
# m_correct=np.arctan2(*a[::-1])
# #create basic scatterplot
# plt.plot(a[0],a[1], 'o', label="x=a[0] | y=a[1]") #x=3 | y=7.6
# # #########################
# # #np.arctan2 
# # numpy.arctan2(x1, x2,[...]
# # Parameters:
              
# # x1
# # array_like, real-valued
# # y-coordinates.

# # x2array_like, real-valued
# # x-coordinates. 
# # ##########################
# plt.axline((0, 0), slope=np.tan(m_false), linewidth=0.5, color='r', label = r'angle in degrees = {}'.format(round(np.rad2deg(m_false),2)))
# plt.axline((0, 0), slope=np.tan(m_correct), linewidth=0.5, color='g', label = r'angle in degrees = {}'.format(round(np.rad2deg(m_correct),2)))
# #plt.plot(a[1][0], m*a[1][0]axline((0, 0), (1, 1), linewidth=4, color='r')+0, '-r', label='y=2x+1')
# plt.legend()
# #add linear regression line to scatterplot 
# #plt.plot(a[1][0], m*a[1][0]+0)
# ax = plt.gca()
# ax.set_xlim([0, 8])
# ax.set_ylim([0, 8])

# Helping functions

In [ ]:
def angle_between(p1, p2):
    ang1 = np.arctan2(*p1[::-1])
    ang2 = np.arctan2(*p2[::-1])
    return np.rad2deg((ang1 - ang2) % (2 * np.pi))
def unit_vector(angle):
    return np.array([np.cos(angle), np.sin(angle)])
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]
def render_ep (boat_plot, index, tw_vec=0, sail_angle=0, target=False):
    from matplotlib import pyplot as plt
    #true wind vector
    #plt.figure(figsize=(10, 6), dpi=200)
    #define plots
    fig, ax = plt.subplots()

    #define coordinates and directions
    if tw_vec.all!=0:
        x = 1
        y = 0.02
        u = tw_vec[0]
        v = tw_vec[1]
    #sail angle   
    if sail_angle.all!=0:
        x1 = 5        
        u1 = sail_angle[0]
        v1 = sail_angle[1]
        sail=ax.quiver(x1, y, u1, v1, color='blue')
        ax.plot([], [], color='blue', label="boat heading")

    #create quiver plot
    wind=ax.quiver(x, y, u, v, color='black')
    ax.plot([], [], color='black', label="wind")

    #starting point
    #target point
    #PENDING: spline connecting each point
    
    boat_plot=np.split(boat_plot,len(boat_plot)/2)
    #boat_plot=np.split(boat_plot,len(boat_plot)/2)
    boat_plot=np.array(boat_plot)
    #for plotting starting point with different symbol
    start=boat_plot[0]
    #for plotting ending point with different symbol
    end=boat_plot[-1]
    #keeping all in between points
    boat_plot=boat_plot[1:-1]
    #boat_plot=boat_plot[0::50]
    # Scatter plot
    #light blue -> darker blue 
    target_plot=env.target_position
    x=boat_plot[:, 0]
    y=boat_plot[:, 1]
    weights = np.arange(1,boat_plot[:, 1].shape[0]+1)
    
    points_size=np.linspace(1,15,len(boat_plot))
    
    #plt.scatter(figsize=(5, 5))
    plt.scatter(start[0], start[1], marker="$A$",  c='green', s=100)
    plt.scatter(end[0], end[1], marker="$B$",  c='red', s=100)
    plt.scatter(x, y, c=weights, cmap='coolwarm' ,s=points_size)
    plt.title(f"{index}-th episode")
    if target==True:
        plt.scatter(target_plot[0], target_plot[1], marker="$T$",  c='green', s=100)
    # Display the plot
    #from matplotlib.pyplot import figure

    ax.legend()
    
    #plt.xlim(2*min(x),2*max(x))
    #plt.ylim(2*min(y),2*max(y))
    #plt.xlim(11,15)
    #plt.ylim(0,0.5)
    plt.show()




In [ ]:
boat_plot=np.split(boat_plot,len(boat_plot)/2)
boat_plot

In [ ]:
np.random.randint(90, size=(1, 2))

# Environment

In [ ]:
import numpy as np

class SailBoatEnvironment:
    def __init__(self, alpha=0.1, gamma=0.9, epsilon=0.9):
        self.step_num=0
        #self.true_wind_direction = np.random.uniform(0, 2*np.pi)
        #PENDING: starting boat pos + target pos kalytera na orizontai sthn init para sthn reset
        #kai h reset na ta pairnei apo tin init
        #fixed start position
        self.boat_position = np.array([2, 3])
        #random start position
        #self.boat_position = np.random.randint(90, size=(1, 2))
        #fixed target position
        self.target_position = np.array([12,15]) 
        #random target position
        #self.target_position = np.random.randint(90, size=(1, 2))
        #to heading einai katheto sto sail, ara an to sail einai stis 90 moires to global x axis, to heading einai stis 0 moires, koitaei pros to +apeiro
        self.sail_configs = np.linspace(np.pi/4, 3*np.pi/4, 5) #angles (in radians) of sail to x global axis

        self.tw_possible_angles=np.linspace(0, np.pi, 8) #angles (in radians) of true wind direction to x global axis
        
        

        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon

    def reset(self):
        #returns
        #i)boat position: starting point of boat        
        #ii)random wind direction, epistrefei to index sto self.tw_possible_angles
        self.step_num=0     
        #random true wind direction
        #self.true_wind_direction = np.random.choice(np.linspace(0, np.pi, 8))
        #fixed true wind direction
        self.true_wind_direction = self.tw_possible_angles[2]
        self.boat_position = np.array([2, 3])
        self.target_position = self.target_position 
        self.tw_velocity_vector=unit_vector(self.true_wind_direction)
        #self.boat_velocity = self.tw_velocity_vector
        #print(f"{self.step_num}: Wind Velocity = {self.tw_velocity_vector}")
        print(self.boat_position)
        return self.boat_position, np.where(self.true_wind_direction==self.tw_possible_angles)[0][0]
        
        
        

#     def get_state(self):
#         relative_wind_direction = self.true_wind_direction - np.arctan2(*self.boat_position[::-1])

#         distance_to_target = np.linalg.norm(self.boat_position - self.target_position)
#         return relative_wind_direction, distance_to_target

    def step(self, action):

        sail_config = action
        self.heading=sail_config #heading = h gwnia twn paniwn se sxesh me ton global x aksona
        #print(f"heading -> {self.heading}")
        #sail_force = sail_config * np.sin(self.true_wind_direction - np.arctan2(*self.boat_position[::-1]))
        #sail_force = sail_config * np.sin(self.true_wind_direction - self.heading)
        #print(f"gwnia: -> {np.sin(self.true_wind_direction - self.heading)} -> sail force={sail_force}")
        #print(f"gwnia? {np.arctan2(*self.boat_position[::-1])}")
        #boat_velocity = np.array([np.cos(self.true_wind_direction), np.sin(self.true_wind_direction)]) * sail_force 
        #scale = 1.3 # for debugging only
        self.heading_vector=unit_vector(sail_config) #unit vector heading
        nominator = self.heading_vector @ self.tw_velocity_vector
        denominator= self.heading_vector @ self.heading_vector
        
        self.boat_velocity = self.heading_vector*nominator*denominator# * scale
        self.boat_position_old=self.boat_position
        self.boat_position = self.boat_position + self.boat_velocity#.astype(np.float64)

        
        self.step_num+=1
        
#         print(f"{self.step_num}: heading_vector={self.heading_vector} | V={np.round(self.boat_velocity,2)} /-/ P={np.round(self.boat_position,2)}\
#         /-/ paragontas={round(nominator*denominator,2)} /-/ Sail Angle to X = {np.rad2deg(np.pi/2+sail_config)} moires\
#         /-/ DF: {np.round(np.linalg.norm(self.boat_position_old-self.boat_position),2)}")

        
        done = False
        truncated = False
        reward = -1


        distance_to_target = np.linalg.norm(self.boat_position - self.target_position)
        if distance_to_target < 1:

            done = True
            reward = 100
        
        if self.step_num >= 1000 or not np.any(np.round(self.boat_velocity,2)):
            truncated = True



        return self.boat_position, reward, done, truncated, self.step_num


In [ ]:
env=SailBoatEnvironment()
#current state = true wind direction + angle of boat to target wrt x axis
#prwto einai gia true wind angles
a=len(env.tw_possible_angles)

#deytero ola ta possible actions
b=len(env.sail_configs)
#q_table = np.random.uniform(low=-2, high=0, size=(a,b))
#q_table
#wd=0,1,2 # wd=1
#dt=0,1 #dt=0 makria, dt=1 konta
#np.argmax(q_table[1,1])
#
#rudder conf = 0:15th column
#np.argmax(q_table[state,0:15])
#sail conf = 16:31th columns
#np.argmax(q_table[state,16:32])

#q_table

# Train Q table

In [ ]:
env=SailBoatEnvironment()
q_table = np.random.uniform(low=-2, high=0, size=(a,b))
EPISODES=100
tw_direction=[]
boat_plot=[]
boat_plot_all_ep=[] #stores all episodes' movement
dones=np.zeros(EPISODES)
for i in range(EPISODES):
    boat_plot=[]
    current_state=env.reset()
    boat_pos=current_state[0]
    tw_direction=np.append(tw_direction,current_state[1])
    done=False
    truncated=False
    step_num=0
    #boat_plot=np.append(boat_plot,np.array(boat_pos))
    while not done: # or not truncated:
        
#         if EPISODES < 500:
#         # Get random action
#             sail_conf_index = random.sample(range(0,len(env.sail_configs)),1)[0]
#         else:
#         # Get "best" action
#             sail_conf_index=np.argmax(q_table[current_state])


        #sail_conf_index=np.argmax(q_table[current_state])
        
        #print(f"{i} ep: sail index->{sail_conf_index}, rudder index->{rudder_conf_index}")
        #H np.argmax epistrefei index, oxi timh, opote pairnoume to index pou mas epistrefei
        #kai blepoume apo ta *_configs poia timh antistoixei se ayto to index

        #action=env.sail_configs[sail_conf_index]
        action=np.deg2rad(231)
        

        
#         if step_num < 7:
#             action=np.deg2rad(15)
#         elif step_num < 15:
#             action=np.deg2rad(216.55)
#         elif step_num < 30:

#             action=np.deg2rad(143)
#         else:
#             action=np.deg2rad(78)
        #action=np.deg2rad(15)

        boat_pos, reward, done, truncated, step_num = env.step(action)
        
        #print(f"step:{step_num}: boat position={boat_pos}")
        
        

        
        boat_plot=np.append(boat_plot,np.array(boat_pos))
        
        

        #Psaxnoume ta Q_max, ena gia to sail setting kai ena gia to rudder setting
        #Sto Q table exoume orisei aythaireta oti oi prwtes 16 columns [0:16] dhladh
        #antistoixoun sto sail setting
        #enw oi ypoloipes 16 columns [16:33] sto rudder config

#         Q_max=np.max(q_table[(current_state)])

#         q_current=q_table[current_state,sail_conf_index]

#         new_q=(1-env.alpha)*q_current+env.alpha*(reward+env.gamma*Q_max)

#         q_table[current_state,sail_conf_index]=new_q

        if done==True:
            print(f"{i}-th ep -> Egina done sto {step_num}-th step.")
            dones[i]=1
            break

        if truncated==True:
        #print(f"{i}-th ep -> Egina truncated")
            break

        

    #print(i)
    boat_plot_all_ep.append(boat_plot)
#     if i%800==0:
#         print(f"{i}-th ep -> steps: {step_num}")
    
#plot last episode
#render_ep(boat_plot, EPISODES)
vec=unit_vector(env.tw_possible_angles[int(tw_direction[i])])

render_ep(boat_plot_all_ep[i],i, vec, np.array([0,0]), True)
#debugging only: constant sail angle
#render_ep(boat_plot_all_ep[i],i, vec, unit_vector(action), True)

In [ ]:
dones
#Count number of successful episodes
num_successful_episodes=np.count_nonzero(dones == 1)
#Index of successful episodes
successful_indices=np.where(dones==1)[0]
print(f"posa epeisodia teleiwsa: {num_successful_episodes} apo ta {i} \ndhladh to {round(num_successful_episodes/i,2)*100}%")

In [ ]:
for i in successful_indices[-10:]:
    render_ep(boat_plot_all_ep[i],i, vec, np.array([0,0]), True)

In [ ]:
for i in sorted(random.sample(range(0,EPISODES),10)):
    render_ep(boat_plot_all_ep[i],i, vec, np.array([0,0]), True)